In [1]:
import numpy as np
import tensorflow as tf
import itertools
import pandas as pd
# Own library
import test2
# Needed to clean memory
import gc

# To modify scipts remotly
%load_ext autoreload
%autoreload 2

In [2]:
np.random.seed(94)
tf.random.set_seed(94)
(_, _), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_test = x_test / 255.0
x_test, y_test = tf.constant(x_test), tf.constant(y_test)

n = 1
x_test = x_test[:n]
y_test = y_test[:n]

In [3]:
path = 'data/model_cifar.h5'
model = tf.keras.models.load_model(path)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [4]:
# Parameter space
list_eta = np.append(np.power(0.1, [1, 2, 3, 4]), 1/255)
list_epsilon = np.append(np.power(0.1, [0, 1, 2, 3, 4]), 1/255)
list_n_steps = np.arange(1, 10, 2)
list_losses = tf.keras.losses.MeanSquaredError()
list_methods = [test2.pgd_infinity, test2.pgd_ininity_random]

# Generate parameters grid
iter_parameters_grid = itertools.product(list_methods, list_eta, list_epsilon, list_n_steps)


# Names of methods
list_method_names = ['pgd_infinity', 'pgd_random']

In [5]:
# Compute different attack sets

nested_accuracies = []
for method, eta, eps, n_steps in iter_parameters_grid:
    if eta >= eps:
        continue
    
    x_adv = method(x_test, y_test, model, loss, eta=eta, eps=eps, n_steps=n_steps)
    # Re-scaling image (they are normalized)
    x_adv = tf.clip_by_value(x_adv, 0, 1)
    list_eval = model.evaluate(x=x_adv, y=y_test)
    # Clear memory
    gc.collect()
    tf.keras.backend.clear_session()
    del x_adv
      
    nested_accuracies.append(list_eval)

1/1 [==============================] - 0s 10ms/sample - loss: 34.2850 - accuracy: 0.0000e+00


1/1 [==============================] - 0s 13ms/sample - loss: 0.1069 - accuracy: 1.0000


1/1 [==============================] - 0s 11ms/sample - loss: 253.6960 - accuracy: 0.0000e+00


In [6]:
index = pd.MultiIndex.from_product([list_method_names, list_eta, list_epsilon],
                                   names=['method', 'tau', 'epsilon'])

In [7]:
# Graphs when step size is big
index = pd.MultiIndex.from_product([list_method_names, list_eta, list_epsilon, list_n_steps],
                                   names=['method', 'eta', 'epsilon', 'n_steps'])

df_results = pd.DataFrame(data=nested_accuracies,
                          columns=['loss', 'accuracy'],
                          index=index)

In [8]:
(df_results.unstack('method').loc[:, ('accuracy','pgd_infinity')]
           .unstack('epsilon')
           .drop(columns=0.0)
           .drop(0.0)
#  .plot(figsize=(16, 9))
)

epsilon      0.2  0.4  0.6  0.8
eta n_steps                    
0.2 1        0.0  0.0  0.0  0.0
    2        0.0  0.0  0.0  0.0
    3        0.0  0.0  0.0  0.0
    4        0.0  0.0  0.0  0.0
0.4 1        0.0  0.0  0.0  0.0
    2        0.0  0.0  0.0  0.0
    3        0.0  0.0  0.0  0.0
    4        0.0  0.0  0.0  0.0
0.6 1        0.0  0.0  0.0  0.0
    2        0.0  0.0  0.0  0.0
    3        0.0  0.0  0.0  0.0
    4        0.0  0.0  0.0  0.0
0.8 1        0.0  0.0  0.0  0.0
    2        0.0  0.0  0.0  0.0
    3        0.0  0.0  0.0  0.0
    4        0.0  0.0  0.0  0.0

In [20]:
path = './data/df_results.pickle'
import pickle
with open(path, 'wb') as f:
    pickle.dump(df_results, f)